In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

## Découper le texte en chunks


In [ ]:
def split_text_into_chunks(text: str, chunk_size: int = 512) -> list:
    """
    Divise un texte en chunks de taille fixe

    Input: 
    -----
        Texte brut.
    Parameters:
    -----------
      chunk_size: Nombre maximum de mots par chunk.
    Returns: 
    -------
        base de données de chunks 
    """
    words = text.split()
    chunks = [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    print(f"Divisé en {len(chunks)} chunks.")
    return chunks


In [5]:
document_text = """
L'intelligence artificielle (IA) est un domaine de l'informatique qui vise à créer des systèmes capables d'exécuter des tâches qui nécessitent normalement une intelligence humaine.
Parmi ces tâches, on trouve la reconnaissance vocale, la vision par ordinateur, la prise de décision et la traduction de langues.

L'IA peut être classée en deux grandes catégories : l'IA faible et l'IA forte. L'IA faible est conçue pour exécuter une tâche spécifique, comme les assistants virtuels (par exemple, Siri ou Alexa).
L'IA forte, quant à elle, a pour objectif de reproduire l'intelligence humaine dans son ensemble, y compris les émotions et la créativité.

Le traitement du langage naturel (NLP) est une sous-discipline importante de l'IA, permettant aux machines de comprendre et de répondre au langage humain.
Grâce à des avancées majeures dans les modèles de langage, comme GPT et BERT, le NLP a considérablement amélioré les performances dans des domaines comme le service client et la traduction automatique.

Enfin, l'IA continue d'évoluer à un rythme rapide, avec des applications dans presque tous les secteurs, de la santé à l'éducation, en passant par la finance et les transports.
"""

chunks = split_text_into_chunks(document_text, chunk_size=512)
print(f"Premier chunk : {chunks[0]}")

Divisé en 1 chunks.
Premier chunk : L'intelligence artificielle (IA) est un domaine de l'informatique qui vise à créer des systèmes capables d'exécuter des tâches qui nécessitent normalement une intelligence humaine. Parmi ces tâches, on trouve la reconnaissance vocale, la vision par ordinateur, la prise de décision et la traduction de langues. L'IA peut être classée en deux grandes catégories : l'IA faible et l'IA forte. L'IA faible est conçue pour exécuter une tâche spécifique, comme les assistants virtuels (par exemple, Siri ou Alexa). L'IA forte, quant à elle, a pour objectif de reproduire l'intelligence humaine dans son ensemble, y compris les émotions et la créativité. Le traitement du langage naturel (NLP) est une sous-discipline importante de l'IA, permettant aux machines de comprendre et de répondre au langage humain. Grâce à des avancées majeures dans les modèles de langage, comme GPT et BERT, le NLP a considérablement amélioré les performances dans des domaines comme le serv

In [ ]:

def generate_embeddings(chunks: list, model_name: str = "paraphrase-multilingual-MiniLM-L12-v2") -> list:
    """
    Génère les embeddings pour une liste de chunks.
    Input :
    ------
        chunks
        model_name
    Parameters
    ---------
        chunks: Liste de chunks de texte.
        model_name: Nom du modèle SentenceTransformer.
    :return: Liste des embeddings.
    """
    print(f"Chargement du modèle {model_name}...")
    model = SentenceTransformer(model_name)
    print("Génération des embeddings...")
    embeddings = model.encode(chunks, batch_size=16, show_progress_bar=True)
    print(f"Embeddings générés pour {len(embeddings)} chunks.")
    return embeddings


In [7]:
embeddings = generate_embeddings(chunks)

Chargement du modèle paraphrase-multilingual-MiniLM-L12-v2...
Génération des embeddings...


Batches: 100%|██████████| 1/1 [00:02<00:00,  2.23s/it]

Embeddings générés pour 1 chunks.


In [ ]:
def create_faiss_index(embeddings: np.ndarray, index_file: str):
    """
    Crée et sauvegarde un index FAISS à partir des embeddings

    Parameters : 
    -----------
        embeddings: Liste ou tableau numpy des embeddings.
        index_file: Chemin pour sauvegarder l'index FAISS.
    Output : 
    ------- 
        base de données 
    """
    dimension = embeddings.shape[1]  # Dimension des vecteurs
    print(f"Création d'un index FAISS avec dimension {dimension}...")
    index = faiss.IndexFlatL2(dimension)  # Index basé sur la distance euclidienne
    index.add(embeddings)
    faiss.write_index(index, index_file)
    print(f"Index FAISS sauvegardé dans {index_file}.")



# Exemple d'utilisation
embeddings_array = np.array(embeddings).astype("float32")
create_faiss_index(embeddings_array, "embeddings/vectors/example_faiss.index")